In [17]:
import pandas as pd

data = pd.read_csv("moocs_stem_reviews.csv")

data.drop(columns=['Unnamed: 1'], inplace=True)
data = data.dropna()
data['Reviews'] = data['Reviews'].astype(str)
data.head()

,Reviews
0,Лучший курс для подготовки к ЕГЭ по математике...
1,Самый лучший курс по профильной математике! Ог...
2,"Замечательный курс для тех, кто хочет самостоя..."
3,Замечательный курс от не менее замечательного ...
4,Лучший курс GOD OF MATH


In [18]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()
    
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

print(data['Reviews'].to_list()[0])
classes = ['Неинформативный', 'Информативный']
print(predict_zero_shot(data['Reviews'].to_list()[0], classes, model, tokenizer))

tqdm.pandas(desc="Processing")
data['is_relevant'] = data['Reviews'].progress_apply(lambda x: np.array(predict_zero_shot(x, classes, model, tokenizer)).argmax())
data['is_relevant'] = data['is_relevant'].astype(int)

Лучший курс для подготовки к ЕГЭ по математике с отличным учителем ever Учёба с Тимуром = СОТОЧКА баллов за экзамен + отлично проведённое время
[0.02235311 0.9776469 ]


Processing: 100%|██████████| 5719/5719 [05:26<00:00, 17.53it/s]


In [19]:
file_path = 'extract_relevant.csv'

# Сохраняем DataFrame в CSV файл
data.to_csv(file_path, index=False)

In [21]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()
    
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

print(data['Reviews'].to_list()[0])
classes = ['Негативный', 'Позитивный']
print(predict_zero_shot(data['Reviews'].to_list()[0], classes, model, tokenizer))

tqdm.pandas(desc="Processing")
data['is_positive'] = data['Reviews'].progress_apply(lambda x: np.array(predict_zero_shot(x, classes, model, tokenizer)).argmax())
data['is_positive'] = data['is_positive'].astype(int)

Лучший курс для подготовки к ЕГЭ по математике с отличным учителем ever Учёба с Тимуром = СОТОЧКА баллов за экзамен + отлично проведённое время
[0.01083749 0.98916256]


Processing: 100%|██████████| 5719/5719 [04:52<00:00, 19.56it/s]


In [22]:
file_path = 'extract_positive_and_relevant.csv'

# Сохраняем DataFrame в CSV файл
data.to_csv(file_path, index=False)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5719 entries, 0 to 5720
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Reviews      5719 non-null   object
 1   is_relevant  5719 non-null   int64 
 2   is_positive  5719 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 178.7+ KB


In [24]:
data.head()

,Reviews,is_relevant,is_positive
0,Лучший курс для подготовки к ЕГЭ по математике...,1,1
1,Самый лучший курс по профильной математике! Ог...,1,1
2,"Замечательный курс для тех, кто хочет самостоя...",1,1
3,Замечательный курс от не менее замечательного ...,1,1
4,Лучший курс GOD OF MATH,1,1


In [25]:
data['is_relevant'].value_counts()

is_relevant
1    5416
0     303
Name: count, dtype: int64

In [27]:
data['is_positive'].value_counts()

is_positive
1    5130
0     589
Name: count, dtype: int64

In [31]:
object_data = pd.read_csv("extract_object.csv")
object_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5719 entries, 0 to 5718
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Reviews  5719 non-null   object
 1   object   5719 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.5+ KB


In [32]:
object_data = pd.read_csv("extract_object.csv")

data['object'] = object_data['object'].astype(int)
file_path = 'stepik_data.csv'

# Сохраняем DataFrame в CSV файл
data.to_csv(file_path, index=False)

In [33]:
data['object'].info()

<class 'pandas.core.series.Series'>
Index: 5719 entries, 0 to 5720
Series name: object
Non-Null Count  Dtype  
--------------  -----  
5717 non-null   float64
dtypes: float64(1)
memory usage: 89.4 KB


In [34]:
data.head()

,Reviews,is_relevant,is_positive,object
0,Лучший курс для подготовки к ЕГЭ по математике...,1,1,1.0
1,Самый лучший курс по профильной математике! Ог...,1,1,1.0
2,"Замечательный курс для тех, кто хочет самостоя...",1,1,1.0
3,Замечательный курс от не менее замечательного ...,1,1,2.0
4,Лучший курс GOD OF MATH,1,1,1.0


In [39]:
data['object'] = data['object'].fillna(1)

In [40]:
data['object'].unique()

array([1., 2., 0.])

In [41]:
data['object'] = data['object'].astype(int)

In [42]:
file_path = 'stepik_data.csv'

# Сохраняем DataFrame в CSV файл
data.to_csv(file_path, index=False)

In [47]:
data = pd.read_csv("stepik_data.csv")
data.tail()

,Reviews,is_relevant,is_positive,object
5714,доступно и понятно все,1,1,1
5715,"шикарнейший курс, во многом помог подготовитьс...",1,1,1
5716,"Курс хороший, но есть местами недочеты.",0,0,1
5717,хороший курс,1,1,1
5718,👍🏻👍🏻👍🏻,1,1,1


In [48]:
data['is_relevant'] = data['is_relevant'] * data['Reviews'].apply(lambda x: len(x.split()) > 8)
data.tail()

,Reviews,is_relevant,is_positive,object
5714,доступно и понятно все,0,1,1
5715,"шикарнейший курс, во многом помог подготовитьс...",1,1,1
5716,"Курс хороший, но есть местами недочеты.",0,0,1
5717,хороший курс,0,1,1
5718,👍🏻👍🏻👍🏻,0,1,1


In [49]:
data['is_relevant'].value_counts()

is_relevant
1    3096
0    2623
Name: count, dtype: int64

In [50]:
file_path = 'stepik_data.csv'

# Сохраняем DataFrame в CSV файл
data.to_csv(file_path, index=False)

# Балансируем объекты и негативные отзывы